In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from urllib.request import urlopen,Request
import requests
import re

In [2]:
file=pd.read_excel('C:/Users/dhanu/AppData/Local/Programs/Python/Python310/BlackCoffe_Assignment/20211030 Test Assignment-20220712T041409Z-001/20211030 Test Assignment/Input.xlsx')

In [3]:
file.head()

,URL_ID,URL
0,1.0,https://insights.blackcoffer.com/is-telehealth...
1,2.0,https://insights.blackcoffer.com/how-telehealt...
2,3.0,https://insights.blackcoffer.com/is-telemedici...
3,4.0,https://insights.blackcoffer.com/is-telehealth...
4,5.0,https://insights.blackcoffer.com/how-people-di...


In [4]:
def get_html(url):
    raw_request = Request(url)
    raw_request.add_header('User-Agent', 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:78.0) Gecko/20100101 Firefox/78.0')
    raw_request.add_header('Accept', 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8')
    resp = urlopen(raw_request)
    raw_html = resp.read()
    soup = BeautifulSoup(raw_html)
    return soup

In [5]:
def get_title(soup):
    title=soup.title
    title_clean=[]
    for x in title:
        title_clean.append(str(x))
    return title_clean
    

In [6]:
def get_text(soup):
    text=soup.find_all("p")
    cont=[]
    
    for x in text:
        cont.append(x.text)
        
    return cont
    

In [ ]:
def save_file(data,file_name):
    
    for i in range(0,data.shape[0]):
        url=data[i]
        html=get_html(url)
        title=get_title(html)
        text=get_text(html)
        text=title+text
        line=str(file_name[i])+str('.txt')
        f = open('%s'%line,mode='x',encoding="utf-8")
        for x in text:
            f.writelines(x)
        f.close()


In [ ]:
save_file(file['URL'],file['URL_ID'])

In [7]:
all_text=[]
for i in range(0,file["URL"].shape[0]):
    url=file["URL"][i]
    html=get_html(url)
    text=get_text(html)
    all_text.append(text)
        

In [8]:
new_data=pd.DataFrame({'URL_ID': file['URL_ID'],
                      'Text': all_text})

In [9]:
neg_words=[]
w=open('negative-words.txt','r')
for x in w :
    neg_words.append(x[:-1])
pos_words=[]
w=open('positive-words.txt','r')
for x in w :
    pos_words.append(x[:-1])
len(neg_words),len(pos_words)   

(4783, 2006)

In [10]:
def scores(text,positive_words,negative_words):
    sum1=""
    for i in text:
        sum1=sum1+i
    sum1=sum1.lower()
    sum1=sum1.split()
    
    pos_score= len([word for word in sum1 if  word in  positive_words])
    neg_score=len([word for word in sum1 if  word in  negative_words])
    return pos_score,neg_score    

In [11]:
pos_score_list=[]
neg_score_list=[]

for i in range(0,file.shape[0]):
    pos_score,neg_score=scores(new_data['Text'][i],pos_words,neg_words)
    pos_score_list.append(pos_score)
    neg_score_list.append(neg_score)
    

In [12]:
output = pd.DataFrame({'URL':file['URL'],
                      'Positive_Score':pos_score_list,
                      'Negative_score':neg_score_list})

In [13]:
output.head()

,URL,Positive_Score,Negative_score
0,https://insights.blackcoffer.com/is-telehealth...,39,5
1,https://insights.blackcoffer.com/how-telehealt...,38,13
2,https://insights.blackcoffer.com/is-telemedici...,70,21
3,https://insights.blackcoffer.com/is-telehealth...,55,15
4,https://insights.blackcoffer.com/how-people-di...,78,25


In [14]:
new_data.head()

,URL_ID,Text
0,1.0,[Covid-19 has paved the way for advancements i...
1,2.0,[The last 2 years are the most critical time i...
2,3.0,"[Telemedicine, which allows patients and docto..."
3,4.0,[Telemedicine refers to a specific set of clin...
4,5.0,"[The future of telehealth stays positive, To a..."


In [15]:
from nltk.corpus import stopwords

In [16]:
def text_measure(text):
    sum1=""
    for i in text:
        sum1=sum1+i
    num_sentence=len(sum1.split('. '))
    text=re.sub('[^a-zA-Z]'," ",sum1)
    text=text.lower()
    text=text.split()
    text= ' '.join([w for w in text if not w in stopwords.words('english')])
    num_words=len(text.split())
    return num_words,num_sentence
    
    

In [17]:
words_list=[]
sentence_list=[]
for i in range (0,new_data.shape[0]):
    words,sentence=text_measure(new_data['Text'][i])
    words_list.append(words)
    sentence_list.append(sentence)
    

In [18]:
output_lengths=pd.DataFrame({
                            'number_of_cleaned_words': words_list,
                            'number_of_sentence':sentence_list,
                            })

In [19]:
output_lengths.head()

,number_of_cleaned_words,number_of_sentence
0,382,14
1,471,43
2,1057,74
3,1127,50
4,1036,60


In [20]:
def count(text):
    sum1=""
    for i in text:
        sum1=sum1+i
    text=re.sub('[^a-zA-Z]'," ",sum1)
    
    text=text.lower()
    text=text.split()
    word_count=0
    n_letter=0
    for i in text:
        word_count+=1
        for x in i:
            n_letter+=1
    return word_count,n_letter

In [21]:
def complex_words(text):
    sum1=""
    for i in text:
        sum1=sum1+i
    text=re.sub('[^a-zA-Z]'," ",sum1)
    text=text.lower()
    text=text.split()
    new=[]
    for i in text:
        if (i[-2:]=='ed')|(i[-2:]=='es'):
            continue
        else:
            new.append(i)
    vowels=['a','e','i','o','u']
    c_word=0
    count=0
    syllabel=0
    for i in new:
        for x in i:
            if x in vowels:
                count+=1
                syllabel+=1
        if count>2:
            c_word+=1
    syllabel=syllabel/len(new)
    return c_word,syllabel

In [22]:
c_word_list=[]
total_syllabel_word=[]
for i in range(0,new_data.shape[0]):
    c_word,syllabel=complex_words(new_data.Text[i])
    c_word_list.append(c_word)
    total_syllabel_word.append(syllabel)
    
        


In [23]:
no_of_words=[]
no_of_characters=[]
for i in range(0,new_data.shape[0]):
    words,letters=count(new_data.Text[i])
    no_of_words.append(words)
    no_of_characters.append(letters)


In [24]:
output_lengths['Complex_word']=c_word_list

In [25]:
output_lengths

,number_of_cleaned_words,number_of_sentence,Complex_word
0,382,14,610
1,471,43,909
2,1057,74,1668
3,1127,50,1686
4,1036,60,1702
...,...,...,...
145,547,40,857
146,637,38,1021
147,652,58,1078
148,321,12,516


In [26]:
output_lengths['Total_word']=no_of_words
output_lengths['Total_characters']=no_of_characters

In [27]:
def pronouns(text):
    sum1=""
    for i in text:
        sum1=sum1+i
    text=re.sub('[^a-zA-Z]'," ",sum1)
    text=text.split()
    new=[]
    for i in text:
        if (i =='US'):
            continue
        else:
            new.append(str(i))
    pronouns=['i','we','my','ours','us']
    for i in range(0,len(new)):
        new[i]=new[i].lower()
    count=0
    for i in new:
        if i in pronouns:
            count+=1
    return count

In [28]:
total_pronouns=[]
for i in range(0,new_data.Text.shape[0]):
    pronouns_count=pronouns(new_data.Text[i])
    total_pronouns.append(pronouns_count)

In [29]:
output_lengths.head()

,number_of_cleaned_words,number_of_sentence,Complex_word,Total_word,Total_characters
0,382,14,610,658,3523
1,471,43,909,955,4640
2,1057,74,1668,1770,9832
3,1127,50,1686,1802,10097
4,1036,60,1702,1810,10002


In [30]:
output_lengths['Polarity_Score']=(output.Positive_Score-output.Negative_score)/((output.Positive_Score+output.Negative_score)+0.000001)
output_lengths['Subjectivity Score']=(output.Positive_Score+output.Negative_score)/(output_lengths.number_of_cleaned_words+0.000001)
output_lengths['Percentage_Complex_words']=output_lengths.Complex_word/output_lengths.number_of_cleaned_words
output_lengths['Fog_Index']=(output_lengths.number_of_sentence+output_lengths['Percentage_Complex_words'])*0.4
output_lengths['Average_Sentence_Length']=output_lengths.number_of_cleaned_words/output_lengths.number_of_sentence
output_lengths['Average_Number_Words_Sentence']=output_lengths.number_of_cleaned_words/output_lengths.number_of_sentence
output_lengths['Average__Words_length']=output_lengths.Total_characters/output_lengths.Total_word
output_lengths['Total_syllables_per_word']=total_syllabel_word
output_lengths['Total_pronouns']=total_pronouns

In [31]:
output_lengths

,number_of_cleaned_words,number_of_sentence,Complex_word,Total_word,Total_characters,Polarity_Score,Subjectivity Score,Percentage_Complex_words,Fog_Index,Average_Sentence_Length,Average_Number_Words_Sentence,Average__Words_length,Total_syllables_per_word,Total_pronouns
0,382,14,610,658,3523,0.772727,0.115183,1.596859,6.238743,27.285714,27.285714,5.354103,1.986907,0
1,471,43,909,955,4640,0.490196,0.108280,1.929936,17.971975,10.953488,10.953488,4.858639,1.814490,0
2,1057,74,1668,1770,9832,0.538462,0.086093,1.578051,30.231220,14.283784,14.283784,5.554802,2.125300,1
3,1127,50,1686,1802,10097,0.571429,0.062112,1.496007,20.598403,22.540000,22.540000,5.603219,2.113879,0
4,1036,60,1702,1810,10002,0.514563,0.099421,1.642857,24.657143,17.266667,17.266667,5.525967,2.078685,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,547,40,857,917,4735,0.041667,0.087751,1.566728,16.626691,13.675000,13.675000,5.163577,1.828471,9
146,637,38,1021,1096,5593,0.571429,0.065934,1.602826,15.841130,16.763158,16.763158,5.103102,1.902153,2
147,652,58,1078,1134,5654,-0.107692,0.099693,1.653374,23.861350,11.241379,11.241379,4.985891,1.879406,2
148,321,12,516,560,3109,0.862069,0.090343,1.607477,5.442991,26.750000,26.750000,5.551786,2.017442,0


In [36]:
final_output=pd.DataFrame({'URL'            : file['URL'],
                           'POSITIVE SCORE' : output.Positive_Score,
                           'NEGATIVE SCORE' : output.Negative_score,
                           'POLARITY SCORE' : output_lengths['Polarity_Score'],
                        'SUBJECTIVITY SCORE': output_lengths['Subjectivity Score'],
                       'AVG SENTENCE LENGTH': output_lengths['Average_Sentence_Length'],
               'PERCENTAGE OF COMPLEX WORDS': output_lengths['Percentage_Complex_words'],
                                 'FOG INDEX': output_lengths['Fog_Index'],
          'AVG NUMBER OF WORDS PER SENTENCE': output_lengths['Average_Number_Words_Sentence'],
                        'COMPLEX WORD COUNT': output_lengths['Complex_word'],
                                'WORD COUNT': output_lengths.number_of_cleaned_words,
                         'SYLLABLE PER WORD': output_lengths['Total_syllables_per_word'],
                         'PERSONAL PRONOUNS': output_lengths['Total_pronouns'],
                           'AVG WORD LENGTH': output_lengths['Average__Words_length']})

In [37]:
final_output

,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,https://insights.blackcoffer.com/is-telehealth...,39,5,0.772727,0.115183,27.285714,1.596859,6.238743,27.285714,610,382,1.986907,0,5.354103
1,https://insights.blackcoffer.com/how-telehealt...,38,13,0.490196,0.108280,10.953488,1.929936,17.971975,10.953488,909,471,1.814490,0,4.858639
2,https://insights.blackcoffer.com/is-telemedici...,70,21,0.538462,0.086093,14.283784,1.578051,30.231220,14.283784,1668,1057,2.125300,1,5.554802
3,https://insights.blackcoffer.com/is-telehealth...,55,15,0.571429,0.062112,22.540000,1.496007,20.598403,22.540000,1686,1127,2.113879,0,5.603219
4,https://insights.blackcoffer.com/how-people-di...,78,25,0.514563,0.099421,17.266667,1.642857,24.657143,17.266667,1702,1036,2.078685,1,5.525967
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,https://insights.blackcoffer.com/blockchain-fo...,25,23,0.041667,0.087751,13.675000,1.566728,16.626691,13.675000,857,547,1.828471,9,5.163577
146,https://insights.blackcoffer.com/the-future-of...,33,9,0.571429,0.065934,16.763158,1.602826,15.841130,16.763158,1021,637,1.902153,2,5.103102
147,https://insights.blackcoffer.com/big-data-anal...,29,36,-0.107692,0.099693,11.241379,1.653374,23.861350,11.241379,1078,652,1.879406,2,4.985891
148,https://insights.blackcoffer.com/business-anal...,27,2,0.862069,0.090343,26.750000,1.607477,5.442991,26.750000,516,321,2.017442,0,5.551786


In [40]:
final_output.to_excel('output.xlsx',index=False)